Sample Data sheet is at
https://docs.google.com/spreadsheets/d/1NCoQiCwjPM42AQRq1qj6UQFv8EKwUeK24eiQ1IChe3I/edit#gid=0

In [1]:
# convert into data from matrix to long format
import pandas as pd
# set pandas to show all columns without truncation and line breaks
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

# data = np.loadtxt('data/test-data.csv', delimiter=',', dtype=int, skiprows=1,)
data = pd.read_csv('data/test-data.csv')
print(data)

# reset the column.index to be numeric
user_index = data[data.columns[0]]
book_index = data.columns
data = data.reset_index(drop=True)
data[data.columns[0]] = data.index.astype('int')
# print(data)
# print(data)
scaler = 5

# data = pd.DataFrame(data.to_numpy(), index=range(0,len(user_index)), columns=range(0,len(book_index)))
df_long = pd.melt(data, id_vars=[data.columns[0]], 
                  ignore_index=True, 
                  var_name='book_id', 
                  value_name='rate').dropna()
df_long.columns = ['user_id', 'book_id', 'rating']
df_long['rating'] = df_long['rating'] / scaler
# replace the user_id to user by match user_index
df_long['user_id'] = df_long['user_id'].apply(lambda x: user_index[x])
# data = df_long.to_numpy()

print(df_long)

    User  Book 1  Book 2  Book 3  Book 4  Book 5  Book 6
0  User1     5.0     2.0     NaN     NaN     NaN     NaN
1  User2     NaN     5.0     NaN     1.0     3.0     2.0
2  User3     NaN     NaN     4.0     NaN     NaN     NaN
3  User4     3.0     1.0     NaN     3.0     NaN     4.0
4  User5     2.0     NaN     1.0     NaN     5.0     2.0
5  User6     1.0     2.0     1.0     NaN     5.0     1.0
6  User7     NaN     1.0     1.0     1.0     NaN     1.0
7  User8     NaN     NaN     NaN     1.0     NaN     NaN
8  User9     1.0     NaN     5.0     NaN     2.0     1.0
   user_id book_id  rating
0    User1  Book 1     1.0
3    User4  Book 1     0.6
4    User5  Book 1     0.4
5    User6  Book 1     0.2
8    User9  Book 1     0.2
9    User1  Book 2     0.4
10   User2  Book 2     1.0
12   User4  Book 2     0.2
14   User6  Book 2     0.4
15   User7  Book 2     0.2
20   User3  Book 3     0.8
22   User5  Book 3     0.2
23   User6  Book 3     0.2
24   User7  Book 3     0.2
26   User9  Book 3     1.

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the MovieLens dataset
# url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
# dataset = pd.read_csv(url, compression='zip', usecols=['userId', 'movieId', 'rating'])
dataset = df_long
# Encode the user and movie IDs
user_encoder = LabelEncoder()
book_encoder = LabelEncoder()
dataset['user_id'] = user_encoder.fit_transform(dataset['user_id'])
dataset['book_id'] = book_encoder.fit_transform(dataset['book_id'])

# Split the dataset into train and test sets
# train, test = train_test_split(dataset, test_size=0.2, random_state=42)
train = dataset

# Model hyperparameters
num_users = len(dataset['user_id'].unique())
num_countries = len(dataset['book_id'].unique())
embedding_dim = 64

# Create the NCF model
inputs_user = tf.keras.layers.Input(shape=(1,))
inputs_book = tf.keras.layers.Input(shape=(1,))
embedding_user = tf.keras.layers.Embedding(num_users, embedding_dim)(inputs_user)
embedding_book = tf.keras.layers.Embedding(num_countries, embedding_dim)(inputs_book)

# Merge the embeddings using concatenation, you can also try other merging methods like dot product or multiplication
merged = tf.keras.layers.Concatenate()([embedding_user, embedding_book])
merged = tf.keras.layers.Flatten()(merged)

# Add fully connected layers
dense = tf.keras.layers.Dense(64, activation='relu')(merged)
dense = tf.keras.layers.Dense(32, activation='relu')(dense)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

# Compile the model
model = tf.keras.Model(inputs=[inputs_user, inputs_book], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])


2023-03-20 09:42:26.734174: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 09:42:27.422544: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-20 09:42:29.571778: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib

In [3]:

# Train the model
model.fit(
    [train['user_id'].values, train['book_id'].values],
    train['rating'].values,
    batch_size=64,
    epochs=100,
    verbose=0,
    # validation_split=0.1,
)


2023-03-20 09:42:40.739477: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x55ed08429630 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-20 09:42:40.739512: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-03-20 09:42:40.803753: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-20 09:42:41.356687: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [5]:
from IPython.display import display, HTML
result_df = {}
for user_i in range(1, 10):
  user = f'User{user_i}'
  result_df[user] = {}
  for book_i in range(1, 7):    
    book = f'Book {book_i}'
    pred_user_id = user_encoder.transform([user])
    pred_book_id = book_encoder.transform([book])
    result = model.predict(x=[pred_user_id, pred_book_id], verbose=0)
    result_df[user][book] = result[0][0]
result_df = pd.DataFrame(result_df).T
result_df *= scaler
out = result_df.style.format("{:.2f}")
display(HTML(out.render()))

/tmp/ipykernel_8020/804887845.py:15: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  display(HTML(out.render()))


,Book 1,Book 2,Book 3,Book 4,Book 5,Book 6
User1,4.76,2.04,4.68,3.71,4.24,4.44
User2,2.48,4.67,4.13,1.11,3.09,1.94
User3,2.16,3.67,4.00,1.65,4.08,2.42
User4,3.04,0.93,4.49,3.02,3.38,3.94
User5,2.01,2.86,1.02,1.32,4.75,1.99
User6,0.99,2.04,0.95,0.78,4.66,1.05
User7,0.94,1.01,1.08,0.75,3.89,1.00
User8,1.89,1.86,2.67,1.06,3.92,1.53
User9,0.90,1.89,4.68,1.16,1.86,1.10


In [ ]:
print(model.weights)